In [92]:
import pandas as pd
import re
#from langchain.llms import OpenAI
import openai
import os
import json 
from langchain_openai import ChatOpenAI

In [93]:
data=pd.read_csv('/home/user/Agents/uc.csv')
data.head() 

,service,subservice_name,subservice_charge,city_name,country_name,source
0,ac_service_repair,Non-Inverter PCB repaired,['₹1800'],ahmedabad,India,UrbanCompany
1,ac_service_repair,Inverter PCB repaired,['₹4000'],ahmedabad,India,UrbanCompany
2,ac_service_repair,Replace LVT,"['₹900', '₹499 (Labour)']",ahmedabad,India,UrbanCompany
3,ac_service_repair,Capacitor 2-5 mfd,"['₹250', '₹349 (Labour)']",ahmedabad,India,UrbanCompany
4,ac_service_repair,Capacitor 10-25 mfd,"['₹400', '₹349 (Labour)']",ahmedabad,India,UrbanCompany


In [94]:

os.environ["OPENAI_API_KEY"] = "" 
openai.api_key = ''

In [95]:
import os
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(verbose=False, temperature=0)

In [96]:

import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np 

# Combine relevant columns for embedding
data['combined'] = data['service'] + " " + data['subservice_name'] + " " + data['subservice_charge'].astype(str)

# Load pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each service category separately
service_categories = data['service'].unique()
faiss_indices = {}

for service in service_categories:
    service_df = data[data['service'] == service].reset_index(drop=True)
    embeddings = np.array(service_df['combined'].apply(lambda x: model.encode(x)).tolist()).astype('float32')
    
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    
    faiss_indices[service] = (index, service_df)

        # Debug: Print the number of entries in each FAISS index
    print(f"Service: {service}, Entries in FAISS index: {index.ntotal}")

Service: ac_service_repair, Entries in FAISS index: 2622
Service: ac_service_repairs, Entries in FAISS index: 1
Service: microwave_repair, Entries in FAISS index: 1505
Service: electricians, Entries in FAISS index: 2209
Service: plumbers, Entries in FAISS index: 2193


In [97]:
def retrieve_service_info(query, service_category, top_k=5):
    """Retrieve service information based on the query and service category."""
    if service_category not in faiss_indices:
      
        return json.dumps([])

    query_embedding = np.array([model.encode(query)]).astype('float32')
    index, service_df = faiss_indices[service_category]
    
    D, I = index.search(query_embedding, top_k)


    valid_indices = [i for i in I[0] if i < len(service_df)]
    
    if not valid_indices:
        return json.dumps([])

    results = service_df.iloc[valid_indices].to_dict(orient='records')


    return json.dumps(results)

In [98]:
from langchain_core.tools import tool
@tool
def get_ac_service_info(query: str, top_k: int = 5) -> str:
    """
    Retrieve AC service information based on the query.

    Args:
    - query (str): The query to search for AC services.
    - top_k (int): Number of top results to return.

    Returns:
    - str: A JSON string containing the top AC service results.
    """
    return retrieve_service_info(query, "ac_service_repair", top_k)


@tool
def get_electrician_info(query: str, top_k: int = 5) -> str:
    """
    Retrieve electrician information based on the query.

    Args:
    - query (str): The query to search for electrician services.
    - top_k (int): Number of top results to return.

    Returns:
    - str: A JSON string containing the top electrician service results.
    """
    return retrieve_service_info(query, "electricians", top_k)

@tool
def get_microwave_repair_info(query: str, top_k: int = 5) -> str:
    """
    Retrieve microwave repair information based on the query.

    Args:
    - query (str): The query to search for microwave repair services.
    - top_k (int): Number of top results to return.

    Returns:
    - str: A JSON string containing the top microwave repair service results.
    """
    return retrieve_service_info(query, "microwave_repair", top_k)

@tool
def get_plumber_info(query: str, top_k: int = 5) -> str:
    """
    Retrieve plumbers information based on the query.

    Args:
    - query (str): The query to search for plumbers.
    - top_k (int): Number of top results to return.

    Returns:
    - str: A JSON string containing the top plumber service results.
    """
    return retrieve_service_info(query, "plumbers", top_k)

In [99]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
# Define the functions as tools
tools = [get_ac_service_info, get_electrician_info, get_microwave_repair_info, get_plumber_info]

# Initialize the agent with the tools
open_ai_agent = initialize_agent(tools,
                                 llm,
                                 agent=AgentType.OPENAI_FUNCTIONS,
                                 verbose=True)

In [100]:
# Function to dynamically route queries
def dynamic_routing(query: str):
    # Use the sentence transformer model to get embeddings for the query
    query_embedding = model.encode([query]).astype('float32')
    
    # Find the closest matching service category
    min_distance = float('inf')
    matched_service = None
    for service, (index, _) in faiss_indices.items():
      
        D, _ = index.search(query_embedding, 1)
        if D[0][0] < min_distance:
            min_distance = D[0][0]
            matched_service = service
    
    return matched_service

In [101]:
from langchain.agents import AgentType
# Initialize the agent
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [102]:
def handle_query(query: str):
    matched_service = dynamic_routing(query)
    
    if matched_service:
        result = open_ai_agent.run(f'get_{matched_service}_info query="{query}"')
    else:
        result = "I don't know."
    
    return result

In [108]:
# Example usage
queries = [
    "I want someone to fix my Western toilet installation (floor mounted) in amritsar under 500"

]

for query in queries:
    result = handle_query(query)
    print(f"Query: {query}\nResult: {result}\n")



> Entering new AgentExecutor chain...

Invoking: `get_plumber_info` with `{'query': 'I want someone to fix my Western toilet installation (floor mounted) in amritsar under 500'}`


[{"service": "plumbers", "subservice_name": "Western toilet installation (floor mounted)", "subservice_charge": "['\u20b91509']", "city_name": "kanpur", "country_name": "India", "source": "UrbanCompany", "combined": "plumbers Western toilet installation (floor mounted) ['\u20b91509']"}, {"service": "plumbers", "subservice_name": "Western toilet installation (floor mounted)", "subservice_charge": "['\u20b91509']", "city_name": "agra", "country_name": "India", "source": "UrbanCompany", "combined": "plumbers Western toilet installation (floor mounted) ['\u20b91509']"}, {"service": "plumbers", "subservice_name": "Western toilet installation (floor mounted)", "subservice_charge": "['\u20b91509']", "city_name": "bhopal", "country_name": "India", "source": "UrbanCompany", "combined": "plumbers Western toilet inst